In [1]:
import sys

In [2]:
!ls 

asennuskomennot.txt		     requirements.txt
convert_tf_checkpoint_to_pytorch.py  runs
data				     RUNS.log
debug_notebook_in_colab.ipynb	     run_tf_ner.py
get_biobert_model.sh		     s800_2
get_s800_data.sh		     scripts
jupyter.3717719.out		     slurm-barebone.sh
jupyter.3726682.out		     slurm-run.sh
jupyter-for-debugging.ipynb	     slurm-run-tmp.sh
latest.err			     slurm-tf-model_converter.sh
latest.out			     tasks.py
launcher.sh			     tasks.py.1
logs				     test.log
models				     utils_ner.py
puhti-install.sh		     utils_ner.py.1
__pycache__			     venv_transformers_ner
README.md


In [3]:
!which python3

/scratch/project_2001426/harttu/july-2020/transformers-ner/venv_transformers_ner/bin/python3


In [4]:
!module load gcc/8.3.0 cuda/10.1.168


Lmod is automatically replacing "intel/19.0.4" with "gcc/8.3.0".


Due to MODULEPATH changes, the following have been reloaded:
  1) hpcx-mpi/2.4.0     2) intel-mkl/2019.0.4



In [5]:
!ls data

dev.txt     s800_2		s800SmallTrain	 s800-tools
labels.txt  s800HalfTrain	s800_tabs	 test.txt
s800	    s800-revision-data	s800_tabs_small  train.txt


In [6]:
MAX_LENGTH="128"
BERT_MODEL="monologg/biobert_v1.0_pubmed_pmc"
BERT_MODEL="./biobert/biobert_v1.0_pubmed_pmc"
BERT_MODEL="monologg/biobert_v1.1_pubmed"
BERT_MODEL="./models/biobertTorch/"
#bert_dir,\
#BERT_MODEL="./"+bert_dir+"/"
OUTPUT_DIR="s800_2"
BATCH_SIZE="32"
NUM_EPOCHS="1"
SAVE_STEPS="750"
SEED="1"
LEARNING_RATE="1e-5"
#  '--config_name', bert_model, \
      
!rm -rf $OUTPUT_DIR    
    
argv = ['run_tf_ner.py','--data_dir','./data/s800SmallTrain/', \
        '--labels', './data/s800SmallTrain/labels.txt',\
        '--model_name_or_path', BERT_MODEL, \
        '--output_dir', OUTPUT_DIR, \
        '--max_seq_length',  MAX_LENGTH,\
        '--num_train_epochs', NUM_EPOCHS, \
        '--learning_rate', "4e-5", \
        '--per_device_train_batch_size', BATCH_SIZE,\
        '--save_steps', SAVE_STEPS,\
        '--seed', SEED,\
        '--learning_rate', LEARNING_RATE,\
        '--do_train','--do_eval','--do_predict',
        '--overwrite_output_dir']

sys.argv = argv

In [7]:
# coding=utf-8
# Copyright 2018 The HuggingFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for named entity recognition."""


import logging
import os
from dataclasses import dataclass, field
from importlib import import_module
from typing import Dict, List, Optional, Tuple

import numpy as np
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    TFAutoModelForTokenClassification,
    TFTrainer,
    TFTrainingArguments,
)
from utils_ner import Split, TFTokenClassificationDataset, TokenClassificationTask


In [8]:


logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    task_type: Optional[str] = field(
        default="NER", metadata={"help": "Task type to fine tune in training (e.g. NER, POS, etc)"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    data_dir: str = field(
        metadata={"help": "The input data dir. Should contain the .txt files for a CoNLL-2003-formatted task."}
    )
    labels: Optional[str] = field(
        metadata={"help": "Path to a file containing all labels. If not specified, CoNLL-2003 labels are used."}
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )

In [9]:

# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TFTrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses()

if (
    os.path.exists(training_args.output_dir)
    and os.listdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

module = import_module("tasks")

try:
    token_classification_task_clazz = getattr(module, model_args.task_type)
    token_classification_task: TokenClassificationTask = token_classification_task_clazz()
except AttributeError:
    raise ValueError(
        f"Task {model_args.task_type} needs to be defined as a TokenClassificationTask subclass in {module}. "
        f"Available tasks classes are: {TokenClassificationTask.__subclasses__()}"
    )


/scratch/project_2001426/harttu/july-2020/transformers-ner/venv_transformers_ner/lib/python3.7/site-packages/transformers/training_args.py:332: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,


In [10]:
training_args

TFTrainingArguments(output_dir='s800_2', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Oct20_11-14-30_r07c49.bullx', logging_first_step=False, logging_steps=500, save_steps=750, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=False, dataloader_drop_last=False, eval_steps=500, dataloader_num_workers=0, past_index=-1, run_name=None, disable_tqdm=False, remove_unused_columns=True, label_names=None, load_best_model_at_end=False, 

In [11]:


# Setup logging
logging.basicConfig(
    filename='test.log',
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(
    "n_replicas: %s, distributed training: %s, 16-bits training: %s",
    training_args.n_replicas,
    bool(training_args.n_replicas > 1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)


In [12]:
# Prepare Token Classification task
labels = token_classification_task.get_labels(data_args.labels)
print("LABELS:")
print(labels)
label_map: Dict[int, str] = {i: label for i, label in enumerate(labels)}
print("LABEL_MAP:")
print(label_map)
num_labels = len(labels)

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=model_args.cache_dir,
)
print("CONFIG")
print(config)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast,
)
#'monologg/biobert_v1.1_pubmed',#
with training_args.strategy.scope():
    model = TFAutoModelForTokenClassification.from_pretrained(
        model_args.model_name_or_path,
        from_pt=True,#bool(".bin" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        
    )

NER - GET_LABELS
	USING:./data/s800SmallTrain/labels.txt
		RETURNING:['B-Species', 'B-Strain', 'I-Species', 'I-Strain', 'O']
LABELS:
['B-Species', 'B-Strain', 'I-Species', 'I-Strain', 'O']
LABEL_MAP:
{0: 'B-Species', 1: 'B-Strain', 2: 'I-Species', 3: 'I-Strain', 4: 'O'}
CONFIG
BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "do_lower_case": false,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-Species",
    "1": "B-Strain",
    "2": "I-Species",
    "3": "I-Strain",
    "4": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-Species": 0,
    "B-Strain": 1,
    "I-Species": 2,
    "I-Strain": 3,
    "O": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model

In [13]:
print("SETTING DATASETS")
# Get datasets
train_dataset = (
    TFTokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.train,
    )
    if training_args.do_train
    else None
)
eval_dataset = (
    TFTokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.dev,
    )
    if training_args.do_eval
    else None
)


SETTING DATASETS


In [14]:



def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != -100:
                out_label_list[i].append(label_map[label_ids[i][j]])
                preds_list[i].append(label_map[preds[i][j]])
    #print("ALIGN_PREDICTIONS:"+str(len(out_label_list)))
    return preds_list, out_label_list

def compute_metrics(p: EvalPrediction) -> Dict:
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)

    return {
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

In [15]:
!ls

asennuskomennot.txt		     README.md
convert_tf_checkpoint_to_pytorch.py  requirements.txt
data				     runs
debug_notebook_in_colab.ipynb	     RUNS.log
get_biobert_model.sh		     run_tf_ner.py
get_s800_data.sh		     scripts
jupyter.3717719.out		     slurm-barebone.sh
jupyter.3726682.out		     slurm-run.sh
jupyter-for-debugging.ipynb	     slurm-run-tmp.sh
latest.err			     slurm-tf-model_converter.sh
latest.out			     tasks.py
launcher.sh			     tasks.py.1
logs				     test.log
models				     utils_ner.py
puhti-install.sh		     utils_ner.py.1
__pycache__			     venv_transformers_ner


In [16]:
import tensorflow as tf

In [17]:
training_args

TFTrainingArguments(output_dir='s800_2', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Oct20_11-14-30_r07c49.bullx', logging_first_step=False, logging_steps=500, save_steps=750, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=False, dataloader_drop_last=False, eval_steps=500, dataloader_num_workers=0, past_index=-1, run_name=None, disable_tqdm=False, remove_unused_columns=True, label_names=None, load_best_model_at_end=False, 

In [18]:
print("INITIALIZING TRAINER")
initial_learning_rate=0.001
decay_steps=10000
# Initialize our Trainer
trainer = TFTrainer(
    model=model,
    #optimizer=(tf.keras.optimizers.SGD(
    #              learning_rate=learning_rate_fn),None),
    #optimizers=(tf.keras.optimizers.Adam(
    #learning_rate=initial_learning_rate, beta_1=0.9, 
    #beta_2=0.999, epsilon=1e-07, amsgrad=False,
    #name='Adam'),tf.keras.optimizers.schedules.PolynomialDecay(
    #initial_learning_rate, decay_steps, 
    #end_learning_rate=0.0001, power=1.0,
    #cycle=False, name=None
    #)),
    args=training_args,
    train_dataset=train_dataset.get_dataset() if train_dataset else None,
    eval_dataset=eval_dataset.get_dataset() if eval_dataset else None,
    compute_metrics=compute_metrics,
)

INITIALIZING TRAINER


In [19]:
vars(trainer)

{'model': <transformers.modeling_tf_bert.TFBertForTokenClassification at 0x7f9dc7d03320>,
 'args': TFTrainingArguments(output_dir='s800_2', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Oct20_11-14-30_r07c49.bullx', logging_first_step=False, logging_steps=500, save_steps=750, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=False, dataloader_drop_last=False, eval_steps=500, dataloader_num_workers=0, past_index=-1, run_name=Non

In [20]:
import tensorflow as tf

In [ ]:

print("TRAINING")
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)


TRAINING
 1/35 [..............................] - ETA: 10s

/scratch/project_2001426/harttu/july-2020/transformers-ner/venv_transformers_ner/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py:493: UserWarning: Using `-1` to mask the loss for the token is deprecated. Please use `-100` instead.
  return py_builtins.overload_of(f)(*args)


 9/35 [======>.......................] - ETA: 9:22

In [ ]:

print("EVALUATING")
# Evaluation
results = {}
if training_args.do_eval:
    logger.info("*** Evaluate ***")

    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")

    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")

        for key, value in result.items():
            logger.info("  %s = %s", key, value)
            writer.write("%s = %s\n" % (key, value))

        results.update(result)

# Predict

In [ ]:

print("PREDICTING")
if training_args.do_predict:
    test_dataset = TFTokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.test,
    )

    predictions, label_ids, metrics = trainer.predict(test_dataset.get_dataset())
    preds_list, labels_list = align_predictions(predictions, label_ids)
    report = classification_report(labels_list, preds_list)

    logger.info("\n%s", report)

    output_test_results_file = os.path.join(training_args.output_dir, "test_results.txt")

    with open(output_test_results_file, "w") as writer:
        writer.write("%s\n" % report)

    # Save predictions
    output_test_predictions_file = os.path.join(training_args.output_dir, "test_predictions.txt")

    with open(output_test_predictions_file, "w") as writer:
        with open(os.path.join(data_args.data_dir, "test.txt"), "r") as f:
            example_id = 0

            for line in f:
                if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                    writer.write(line)

                    if not preds_list[example_id]:
                        example_id += 1
                elif preds_list[example_id]:
                    output_line = line.split()[0] + " " + preds_list[example_id].pop(0) + "\n"

                    writer.write(output_line)
                else:
                    logger.warning("Maximum sequence length exceeded: No prediction for '%s'.", line.split()[0])



In [ ]:
tmp = os.path.join(data_args.data_dir, "test.txt")
!head -20 $tmp

In [ ]:
tmp = os.path.join(training_args.output_dir, "test_predictions.txt")
!head -20 $tmp